# 1. Loading libraries and functions


In [ ]:
%load_ext nb_black
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
sns.set()

import seaborn as sns


pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 200)
plt.rcParams.update({'font.family' : 'normal',
        'font.weight' : 'bold','font.size': 22})

In [ ]:
# function to plot a boxplot and a histogram along the same scale.
def histogram_boxplot(data, feature, figsize=(12, 7), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to the show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a star will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins, palette="winter"
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

# function to create labeled barplots



def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 1, 5))
    else:
        plt.figure(figsize=(n + 1, 5))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n].sort_values(),
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot
    
def treat_outliers(df, col,whisk=1.5):
    
    """
    Treats outliers in a variable

    df: dataframe
    col: dataframe column
    whisk:stader=1.5 but you can change it
    """
    Q1 = df[col].quantile(0.25)  # 25th quantile
    Q3 = df[col].quantile(0.75)  # 75th quantile
    IQR = Q3 - Q1
    Lower_Whisker = Q1 - whisk * IQR
    Upper_Whisker = Q3 + whisk * IQR

    # all the values smaller than Lower_Whisker will be assigned the value of Lower_Whisker
    # all the values greater than Upper_Whisker will be assigned the value of Upper_Whisker
    df[col] = np.clip(df[col], Lower_Whisker, Upper_Whisker)

    return df


def treat_outliers_all(df, col_list,whisk=1.5):
    """
    Treat outliers in a list of variables

    df: dataframe
    col_list: list of dataframe columns
    """
    for c in col_list:
        df = treat_outliers(df, c,whisk=1.5)

    return df
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf
def treating_multicollinearity(predictors, target, high_vif_columns):
    """
    Checking the effect of dropping the columns showing high multicollinearity
    on model performance (adj. R-squared and RMSE)

    predictors: independent variables
    target: dependent variable
    high_vif_columns: columns having high VIF
    """
    # empty lists to store adj. R-squared and RMSE values
    adj_r2 = []
    rmse = []

    # build ols models by dropping one of the high VIF columns at a time
    # store the adjusted R-squared and RMSE in the lists defined previously
    for cols in high_vif_columns:
        # defining the new train set
        train = predictors.loc[:, ~predictors.columns.str.startswith(cols)]

        # create the model
        olsmodel = sm.OLS(target, train).fit()

        # adding adj. R-squared and RMSE to the lists
        adj_r2.append(olsmodel.rsquared_adj)
        rmse.append(np.sqrt(olsmodel.mse_resid))

    # creating a dataframe for the results
    temp = pd.DataFrame(
        {
            "col": high_vif_columns,
            "Adj. R-squared after_dropping col": adj_r2,
            "RMSE after dropping col": rmse,
        }
    ).sort_values(by="Adj. R-squared after_dropping col", ascending=False)
    temp.reset_index(drop=True, inplace=True)

    return temp


# 2. Loading and exploring the data



In [ ]:
df = pd.read_csv("insurance.csv")
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns.')  
np.random.seed(1)
df.sample(n=10)


In [ ]:
df.info()  # down to 82 columns after the initial 88

In [ ]:
# looking at which columns have the most missing values
df.isnull().sum().sum()

No missing values

In [ ]:
df.describe(include='all').T


# Univariate analysis 

In [ ]:
histogram_boxplot(df,'age')

In [ ]:
histogram_boxplot(df,'bmi')

In [ ]:
histogram_boxplot(df,'charges')

In [ ]:
labeled_barplot(df,'smoker',perc=True)

In [ ]:
labeled_barplot(df,'region',perc=True)

In [ ]:
labeled_barplot(df,'children',perc=True)

# Bivariate Analysis

In [ ]:
sns.pairplot(data=df.select_dtypes('number'))

In [ ]:
df.groupby(by='region').median()

In [ ]:
sns.barplot(data=df,y='charges',x='region')

In [ ]:
sns.barplot(data=df,y='bmi',x='children')

In [ ]:
sns.barplot(data=df,y='bmi',x='region')

In [ ]:
sns.heatmap(df.corr(),cmap='Spectral_r',annot=True)

In [ ]:
# looking at value counts for non-numeric features

num_to_display = 10  # defining this up here so it's easy to change later if I want
for colname in df.dtypes[df.dtypes == 'object'].index:
    val_counts = df[colname].value_counts(dropna=False)  # i want to see NA counts
    print(val_counts[:num_to_display])
    if len(val_counts) > num_to_display:
        print(f'Only displaying first {num_to_display} of {len(val_counts)} values.')
    print('\n\n') # just for more space between 

# Hypothesis Testing 

In [ ]:

from scipy.stats import f_oneway,ttest_ind,chi2_contingency 

In [ ]:
sns.boxplot(data=df,y='charges',x='children')

In [ ]:
ch0=df[df['children']==0]['charges']
ch1=df[df['children']==1]['charges']
ch2=df[df['children']==2]['charges'] 
ch3=df[df['children']==3]['charges']
ch4=df[df['children']==4]['charges']
ch5=df[df['children']==5]['charges']

In [ ]:
stats,pvalue=f_oneway(ch0,ch1,ch2,ch3,ch4,ch5)
print("tstat = ", stats, ", p-value = ", pvalue)

Since p-value(0.003) < 0.05 (alpha), we reject the null hypothesis and conclude that atleast one of the mean charges in the six children category is unequal.

In [ ]:
female=df[df['sex']=='female']['charges']
male=df[df['sex']=='male']['charges']

In [ ]:
t,  p_value = ttest_ind(male, female)
print("tstat = ",t, ", p_value = ", p_value)

Since p-value(0.055) > 0.05 (alpha), we failed to reject the null hypothesis and conclude that there is no a difference in the two ( male and female) charges means.

In [ ]:
cross=pd.crosstab(df.sex,df.region)
chi2, pval, dof, exp_freq=chi2_contingency(cross)
pval


Since p-value(0.9) > 0.05 (alpha), we failed to reject the null hypothesis and conclude that the two features(sex,region) are independent. 

In [ ]:
cross=pd.crosstab(df.smoker,df.region)
chi2, pval, dof, exp_freq=chi2_contingency(cross)
pval


Since p-value(0.06) > 0.05 (alpha), we failed to reject the null hypothesis and conclude that the two features(smoker,region) are independent. 

In [ ]:
cross=pd.crosstab(df.smoker,df.sex)
chi2, pval, dof, exp_freq=chi2_contingency(cross)
pval

Since p-value(0.006) < 0.05 (alpha), we reject the null hypothesis and conclude that the two features(smoker,sex) are not independent. 

In [ ]:
s_no=df[df['smoker']=='no']['charges']
s_yes=df[df['smoker']=='yes']['charges']
t, p_value = ttest_ind(s_no, s_yes)
print("tstat = ",t, ", p_value = ", p_value)

Since p-value(0.000008) < 0.05 (alpha), we failed to reject the null hypothesis and conclude that the mean fof the two featurs are not equal. the smoker and non smoker has different charges 

In [ ]:
region_se=df[df['region']=='southeast']['charges']
region_sw=df[df['region']=='southwest' ]['charges']   
region_nw=df[df['region']=='northwest' ]['charges']   
region_ne=df[df['region']=='northeast' ]['charges']  
t, p_value = f_oneway(region_se,region_sw,region_nw,region_ne)
print("tstat = ",t, ", p_value = ", p_value)

Since p-value(0.07) > 0.05 (alpha), we failed to reject the null hypothesis and conclude that there is no a difference in the two means.

# Outlier Treatment

In [ ]:
treat_out_cols=df.skew()[df.skew()>1].index.tolist()
print(treat_out_cols)
df11 = treat_outliers_all(df, treat_out_cols,whisk=3)


In [ ]:
df22=df11.astype({'children':'category'})
df22['bmi']=np.log(df.bmi+1)

# One hot encoding 

In [ ]:
df1=pd.get_dummies(df22,drop_first=True)
df1

# scaling 

In [ ]:
df2=df1.copy()


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
X=df2.drop('charges',axis=1)
y=df2['charges']

scale=MinMaxScaler()
X=pd.DataFrame(scale.fit_transform(X),columns=X.columns.tolist())
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=1)

# Ordinary Linear Regression 

In [ ]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [ ]:
print(f'{X_train.shape},{X_test.shape}')

print(f'{y_train.shape},{y_test.shape}')


In [ ]:
ols=sm.OLS(y_train,X_train)
olsfit=ols.fit()
olsfit.summary()

# Linear model assumptions 

Collinearity 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# we will define a function to check VIF
def checking_vif(predictors):
    vif = pd.DataFrame()
    vif["feature"] = predictors.columns

    # calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(predictors.values, i)
        for i in range(len(predictors.columns))
    ]
    return vif

In [ ]:


checking_vif(X_train) 

There are no VIF values higher than 5. 

In [ ]:


# initial list of columns
cols = X_train.columns.tolist()   

# setting an initial max p-value
max_p_value = 1

while len(cols) > 0:
    # defining the train set
    x_train_aux = X_train[cols]   

    # fitting the model
    model = sm.OLS(y_train, x_train_aux).fit()

    # getting the p-values and the maximum p-value
    p_values = model.pvalues.drop('const')
    max_p_value = max(p_values)

    # name of the variable with maximum p-value
    feature_with_p_max = p_values.idxmax()
    #print(feature_with_p_max) 
    if max_p_value > 0.05:

        cols.remove(feature_with_p_max)
    else:
        break

selected_features = cols


print(selected_features)




In [ ]:
X_train2 = X_train[selected_features]

X_test2 = X_test[selected_features]

In [ ]:
olsmod = sm.OLS(y_train, X_train2)
olsres = olsmod.fit()
olsres.summary()

In [ ]:
df_pred=pd.DataFrame({'fitted':olsres.fittedvalues,'actual':y_train,'res':olsres.resid})

In [ ]:
model_performance_regression(olsres,X_train2,y_train)

In [ ]:
model_performance_regression(olsres,X_test2,y_test)

Test homoscedasticity  

In [ ]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(olsres.resid,olsres.model.exog) ## Complete the code to apply the Goldfeldquandt test
lzip(name, test)

In [ ]:
sns.residplot(
    data=df_pred, x="fitted", y="res", color="purple", lowess=True
)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Fitted vs Residual plot")
plt.show()


TEST FOR NORMALITY




In [ ]:


sns.histplot(data=df_pred,x='res') ## Complete the code to plot the distribution of residuals
plt.title("Normality of residuals")
plt.show()



In [ ]:
import pylab
import scipy.stats as stats

stats.probplot(df_pred.res, dist="norm", plot=pylab) ## Complete the code check Q-Q plot
plt.show()



In [ ]:
stats.shapiro(df_pred.res.values)

The assumptions of linear regression are not valied. 

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv =6, 
                               verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
model_performance_regression( rf_random.best_estimator_,X_train,y_train)

In [ ]:
model_performance_regression(rf_random.best_estimator_,X_test,y_test)

In [ ]:
gg=rf_random.best_estimator_.fit(X_train,y_train).feature_importances_

gg


In [ ]:
feature_importance=pd.DataFrame(gg,index=X_train.columns.tolist(),columns=['coef'])

feature_importance.sort_values(by='coef',ascending=False).plot.bar()

In [ ]:
feature_3=rf_random.best_estimator_.fit(X_train[['age','bmi','smoker_yes']],y_train)

In [ ]:
feature_3.score(X_train[['age','bmi','smoker_yes']],y_train)

In [ ]:
feature_3.score(X_test[['age','bmi','smoker_yes']],y_test)